# COIN-OR: MILP (Gemischt-ganzzahlige Optimierung)

In [2]:
import json
import pulp
import pandas as pd

In [20]:
# Basics
import utils.basics.converter as convert
import utils.basics.presenter as show

# Schedule
from utils.checker import is_machine_conflict_free
from utils.checker import is_job_machine_sequence_correct

In [22]:
# Datei laden
with open("data/jobshop_instances.json", "r", encoding="utf-8") as f:
    jobshop_instances = json.load(f)
instance =  jobshop_instances["instance ft10"]
show.print_jobs(instance) 

job 0:  [[0, 29], [1, 78], [2, 9], [3, 36], [4, 49], [5, 11], [6, 62], [7, 56], [8, 44], [9, 21]]
job 1:  [[0, 43], [2, 90], [4, 75], [9, 11], [3, 69], [1, 28], [6, 46], [5, 46], [7, 72], [8, 30]]
job 2:  [[1, 91], [0, 85], [3, 39], [2, 74], [8, 90], [5, 10], [7, 12], [6, 89], [9, 45], [4, 33]]
job 3:  [[1, 81], [2, 95], [0, 71], [4, 99], [6, 9], [8, 52], [7, 85], [3, 98], [9, 22], [5, 43]]
job 4:  [[2, 14], [0, 6], [1, 22], [5, 61], [3, 26], [4, 69], [8, 21], [7, 49], [9, 72], [6, 53]]
job 5:  [[2, 84], [1, 2], [5, 52], [3, 95], [8, 48], [9, 72], [0, 47], [6, 65], [4, 6], [7, 25]]
job 6:  [[1, 46], [0, 37], [3, 61], [2, 13], [6, 32], [5, 21], [9, 32], [8, 89], [7, 30], [4, 55]]
job 7:  [[2, 31], [0, 86], [1, 46], [5, 74], [4, 32], [6, 88], [8, 19], [9, 48], [7, 36], [3, 79]]
job 8:  [[0, 76], [1, 69], [3, 76], [5, 51], [2, 85], [9, 11], [6, 40], [7, 89], [4, 26], [8, 74]]
job 9:  [[1, 85], [0, 13], [2, 61], [6, 7], [8, 64], [9, 76], [5, 47], [3, 52], [4, 90], [7, 45]]



In [24]:
df_instance = convert.jssp_dict_to_df(instance)
df_instance

,Job,Operation,Machine,Processing Time
0,job 0,0,M0,29
1,job 0,1,M1,78
2,job 0,2,M2,9
3,job 0,3,M3,36
4,job 0,4,M4,49
...,...,...,...,...
95,job 9,5,M9,76
96,job 9,6,M5,47
97,job 9,7,M3,52
98,job 9,8,M4,90


In [56]:
solver_time_limit_min = 4

## Solver
- CBC (COIN-OR Branch and Cut)
- HiGHS (high performance serial and parallel solver)

In [36]:
# conda install -c conda-forge highs

In [52]:
def solve_jobshop(df_jssp: pd.DataFrame,
                  solver: str = 'CBC',
                  solver_time_limit: int = 300,
                  epsilon: float = 0.0):
    """
    Minimiert den Makespan eines Job-Shop-Problems auf Basis eines DataFrames.

    Parameter:
    - df_jssp: DataFrame mit Spalten ['Job','Operation','Machine','Processing Time'].
    - solver:  'CBC' oder 'HiGHS' (wenigstens eine dieser beiden Optionen).
    - solver_time_limit: Zeitlimit in Sekunden.
    - epsilon: Puffer in Minuten zwischen Operationen auf derselben Maschine.

    Rückgabe:
    - df_schedule: DataFrame mit Spalten
      ['Job','Operation','Machine','Start','Processing Time','End']
    - makespan_value: minimaler Makespan (float)
    """
    # 1) Index und Op-Nummer
    df = df_jssp.reset_index(drop=False).rename(columns={'index':'Idx'}).copy()
    df['OpIdx'] = df.groupby('Job')['Operation'].first()  # nur zur Konsistenz

    # 2) LP-Modell
    prob = pulp.LpProblem('JSSP', pulp.LpMinimize)
    starts = {
        idx: pulp.LpVariable(f'start_{idx}', lowBound=0, cat='Continuous')
        for idx in df['Idx']
    }
    makespan = pulp.LpVariable('makespan', lowBound=0, cat='Continuous')
    prob += makespan

    # 3) Reihenfolge je Job
    for job, group in df.groupby('Job', sort=False):
        seq = group.sort_values('Operation')
        for prev, curr in zip(seq['Idx'], seq['Idx'][1:]):
            dur_prev = df.loc[df['Idx'] == prev, 'Processing Time'].iat[0]
            prob += starts[curr] >= starts[prev] + dur_prev

    # 4) Maschinenkonflikte
    M = 1e6
    for _, group in df.groupby('Machine', sort=False):
        ids = group['Idx'].tolist()
        for i in range(len(ids)):
            for j in range(i+1, len(ids)):
                i_idx, j_idx = ids[i], ids[j]
                if df.loc[df['Idx'] == i_idx, 'Job'].iat[0] == df.loc[df['Idx'] == j_idx, 'Job'].iat[0]:
                    continue
                di = df.loc[df['Idx'] == i_idx, 'Processing Time'].iat[0]
                dj = df.loc[df['Idx'] == j_idx, 'Processing Time'].iat[0]
                y = pulp.LpVariable(f'y_{i_idx}_{j_idx}', cat='Binary')
                prob += starts[i_idx] + di + epsilon <= starts[j_idx] + M*(1-y)
                prob += starts[j_idx] + dj + epsilon <= starts[i_idx] + M*y

    # 5) Makespan-Constraints
    for _, row in df.iterrows():
        idx = int(row['Idx'])
        prob += starts[idx] + row['Processing Time'] <= makespan

    # 6) Solver auswählen und ausführen
    if solver.upper() in ['CBC', 'BRANCH AND CUT']:
        cmd = pulp.PULP_CBC_CMD(msg=True, timeLimit=solver_time_limit)
    elif solver.upper() == 'HIGHS':
        cmd = pulp.HiGHS_CMD(msg=True, timeLimit=solver_time_limit)
    else:
        raise ValueError("Solver must be 'CBC' or 'HiGHS'")
    prob.solve(cmd)

    # 7) Ergebnis aufbereiten
    df['Start'] = df['Idx'].map(lambda i: round(starts[i].varValue, 2))
    df['End']   = df['Start'] + df['Processing Time']
    df_schedule = df[['Job','Operation','Machine','Start','Processing Time','End']]\
                  .sort_values(['Start','Job','Operation'])\
                  .reset_index(drop=True)
    makespan_value = round(pulp.value(makespan), 3)

    return df_schedule, makespan_value


### a) CBC

In [58]:
df_schedule_cbc, makespan = solve_jobshop(df_instance, 
                                          solver = "Branch and Cut",
                                          solver_time_limit=solver_time_limit_min*60)
df_schedule_cbc

,Job,Operation,Machine,Start,Processing Time,End
0,job 3,0,M1,0.0,81,81.0
1,job 7,0,M2,0.0,31,31.0
2,job 8,0,M0,0.0,76,76.0
3,job 5,0,M2,41.0,84,125.0
4,job 2,0,M1,81.0,91,172.0
...,...,...,...,...,...,...
95,job 9,9,M7,1063.0,45,1108.0
96,job 6,9,M4,1078.0,55,1133.0
97,job 0,9,M9,1079.0,21,1100.0
98,job 1,9,M8,1079.0,30,1109.0


In [46]:
print(f"Makespan: {makespan}\n")

category = "Branch and Cut"
show.plot_gantt_jobs(df_schedule_cbc, 'Gantt-Diagramm für "{}"'.format(category))
show.plot_gantt_machines(df_schedule_cbc, 'Gantt-Diagramm für "{}"'.format(category))

is_machine_conflict_free(df_schedule_cbc)
is_job_machine_sequence_correct(df_schedule_cbc, instance)

1133.0

### b) HiGHS mit PuLP

In [64]:
df_schedule_highs, makespan = solve_jobshop(df_instance, 
                                          solver = "HiGHS",
                                          solver_time_limit=solver_time_limit_min*60)

In [ ]:
print(f"Makespan: {makespan}\n")

category = "HiGHS"
show.plot_gantt_jobs(df_schedule_highs, 'Gantt-Diagramm für "{}"'.format(category))
show.plot_gantt_machines(df_schedule_highs, 'Gantt-Diagramm für "{}"'.format(category))

is_machine_conflict_free(df_schedule_highs)
is_job_machine_sequence_correct(df_schedule_highs, instance)

In [ ]:
df_schedule_highs.to_csv('data/01_schedule_highs.csv', index=False)